In [2]:
import pandas as pd




In [3]:
# Loading the dataset into a dataframe
heart_df = pd.read_csv('heart.csv')

# Displaying the first few rows of the dataframe
heart_df.head()

age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0

In [4]:
# Identifying categorical columns based on unique values
cateogorycolumns = [col for col in heart_df.columns if heart_df[col].nunique() < 15 and col != 'target']

# Using label encoding on identified categorical columns
from sklearn.preprocessing import LabelEncoder

encodedvalue = heart_df.copy()
label_encoders = {}

for col in cateogorycolumns:
    le = LabelEncoder()
    encodedvalue[col] = le.fit_transform(encodedvalue[col])
    label_encoders[col] = le

encodedvalue.head()


age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0

In [5]:
from sklearn.model_selection import train_test_split

# Splitting the data into features and target
X = encodedvalue.drop('target', axis=1)
y = encodedvalue['target']

# Splitting the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


((820, 13), (205, 13), (820,), (205,))

In [6]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Initializing the model
model = LogisticRegression(max_iter=10000)

# Using RFE for feature selection
rfe = RFE(model, n_features_to_select=1)
fit = rfe.fit(X_train, y_train)

# Displaying the ranking of features
featurerank = pd.DataFrame({
    'Feature': X.columns,
    'Ranking': fit.ranking_
}).sort_values(by='Ranking')

featurerank


Feature  Ranking
8      exang        1
12      thal        2
1        sex        3
9    oldpeak        4
2         cp        5
11        ca        6
10     slope        7
6    restecg        8
5        fbs        9
7    thalach       10
3   trestbps       11
4       chol       12
0        age       13

In [ ]:
# Notes 
# exang, thal, and sex are the top three features that RFE suggests as most relevant.

In [9]:
# Applying the forward selection on that 

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Forward selection
sfs = SFS(model, 
                  k_features='best', 
                  forward=True, 
                  floating=False, 
                  scoring='accuracy',
                  cv=5)

sfs = sfs.fit(X_train, y_train)
selected_features_forward = sfs.k_feature_names_



In [12]:
# Backward selection
sfs_backward = SFS(model, 
                   k_features=1, 
                   forward=False, 
                   floating=False, 
                   scoring='accuracy',
                   cv=5)

sfs_backward = sfs_backward.fit(X_train, y_train)
selected_features_backward = sfs_backward.k_feature_names_


In [10]:
selected_features_forward

('age', 'sex', 'chol', 'thalach', 'exang', 'oldpeak', 'ca')

In [13]:
selected_features_backward

('age',)

In [14]:
# use Sequential Feature Removal (or Addition) to figure out right no features to predict  

from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest model
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Sequential Feature Addition (Forward Selection)
sfs = SFS(model_rf, 
                  k_features='best', 
                  forward=True, 
                  floating=False, 
                  scoring='accuracy',
                  cv=5)

sfs = sfs.fit(X_train, y_train)

# Sequential Feature Removal (Backward Selection)
sfs_backward = SFS(model_rf, 
                   k_features=1, 
                   forward=False, 
                   floating=False, 
                   scoring='accuracy',
                   cv=5)

sfs_backward = sfs_backward.fit(X_train, y_train)

sfs_forward_results = {
    'Number of Features': list(range(1, len(sfs.subsets_) + 1)),
    'Features': [sfs.subsets_[i]['feature_names'] for i in range(1, len(sfs.subsets_) + 1)],
    'CV Score': [sfs.subsets_[i]['avg_score'] for i in range(1, len(sfs.subsets_) + 1)]
}

sfs_backward_results = {
    'Number of Features': list(range(1, len(sfs_backward.subsets_) + 1)),
    'Features': [sfs_backward.subsets_[i]['feature_names'] for i in range(1, len(sfs_backward.subsets_) + 1)],
    'CV Score': [sfs_backward.subsets_[i]['avg_score'] for i in range(1, len(sfs_backward.subsets_) + 1)]
}

sfs_forward_best_features = sfs.k_feature_names_
sfs_backward_best_features = sfs_backward.k_feature_names_

sfs_forward_best_features, sfs_backward_best_features


(('age', 'sex', 'chol', 'thalach', 'exang', 'oldpeak', 'ca'), ('age',))

In [ ]:
# # Notes 
# ('age', 'sex', 'chol', 'thalach', 'exang', 'oldpeak', 'ca'),\

#  	   exang 	1
# 12 	thal 	2
# 1 	sex 	3
# 9 	oldpeak 	4
# 2 	cp 	5
# 11 	ca 	6
# 10 	slope 	7

In [19]:
# #
# train  the model using logistic regression , svm , random forest and naive bayes
# including  with all features and only use features ('age', 'sex', 'chol', 'thalach', 'exang', 'oldpeak', 'ca') .draw a table by using performance 
# metrics accuracy , f1 score , precision, recall ,ROC and AUC and also provide me the confusion matrix for both the cases .
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
models = {
    'Logistic Regression': LogisticRegression(max_iter=10000, random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Naive Bayes': GaussianNB()
}

In [21]:
# Selecting specified features
from sklearn.model_selection import cross_val_predict
selected_features = ['age', 'sex', 'chol', 'thalach', 'exang', 'oldpeak', 'ca']
X_train_selected = X_train[selected_features]

# For storing results for the selected features
metrics_selected_features = {}
confusion_matrices_selected_features = {}

# Training models and collecting metrics using the selected features
for name, model in models.items():
    y_pred = cross_val_predict(model, X_train_selected, y_train, cv=5)
    
    accuracy = accuracy_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred)
    recall = recall_score(y_train, y_pred)
    auc = roc_auc_score(y_train, y_pred)
    
    metrics_selected_features[name] = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Precision': precision,
        'Recall': recall,
        'AUC': auc
    }
    
    confusion_matrices_selected_features[name] = confusion_matrix(y_train, y_pred)

metrics_selected_features, confusion_matrices_selected_features


({'Logistic Regression': {'Accuracy': 0.7975609756097561,
   'F1 Score': 0.8074245939675174,
   'Precision': 0.7927107061503417,
   'Recall': 0.8226950354609929,
   'AUC': 0.7967379459420834},
  'SVM': {'Accuracy': 0.6951219512195121,
   'F1 Score': 0.7171945701357466,
   'Precision': 0.6876355748373102,
   'Recall': 0.7494089834515366,
   'AUC': 0.6933442902144331},
  'Random Forest': {'Accuracy': 0.9841463414634146,
   'F1 Score': 0.9846153846153846,
   'Precision': 0.985781990521327,
   'Recall': 0.983451536643026,
   'AUC': 0.9841690932585406},
  'Naive Bayes': {'Accuracy': 0.7975609756097561,
   'F1 Score': 0.8083140877598153,
   'Precision': 0.7900677200902935,
   'Recall': 0.8274231678486997,
   'AUC': 0.7965831204482793}},
 {'Logistic Regression': array([[306,  91],
         [ 75, 348]], dtype=int64),
  'SVM': array([[253, 144],
         [106, 317]], dtype=int64),
  'Random Forest': array([[391,   6],
         [  7, 416]], dtype=int64),
  'Naive Bayes': array([[304,  93],
     

In [15]:
from sklearn.metrics import roc_curve

# Variables to store performance metrics and confusion matrices
performance_metrics_all = {}
confusion_all = {}

performance_metrics_selected = {}
confusion_selected = {}

X_test_selected = X_test[selected_features]

# Training and evaluating models using all features
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]
    
    # Performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc = roc_auc_score(y_test, y_prob)
    
    performance_metrics_all[name] = [accuracy, f1, precision, recall, auc]
    confusion_all[name] = confusion_matrix(y_test, y_pred)

# Training and evaluating models using selected features
for name, model in models.items():
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    y_prob = model.predict_proba(X_test_selected)[:, 1]
    
    # Performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc = roc_auc_score(y_test, y_prob)
    
    performance_metrics_selected[name] = [accuracy, f1, precision, recall, auc]
    confusion_selected[name] = confusion_matrix(y_test, y_pred)

performance_metrics_all, confusion_all, performance_metrics_selected, confusion_selected


NameError: name 'selected_features' is not defined

In [22]:
import pandas as pd

# Provided data
metrics = {
    'Logistic Regression': {'Accuracy': 0.7975609756097561,
                            'F1 Score': 0.8074245939675174,
                            'Precision': 0.7927107061503417,
                            'Recall': 0.8226950354609929,
                            'AUC': 0.7967379459420834},
    'SVM': {'Accuracy': 0.6951219512195121,
            'F1 Score': 0.7171945701357466,
            'Precision': 0.6876355748373102,
            'Recall': 0.7494089834515366,
            'AUC': 0.6933442902144331},
    'Random Forest': {'Accuracy': 0.9841463414634146,
                      'F1 Score': 0.9846153846153846,
                      'Precision': 0.985781990521327,
                      'Recall': 0.983451536643026,
                      'AUC': 0.9841690932585406},
    'Naive Bayes': {'Accuracy': 0.7975609756097561,
                    'F1 Score': 0.8083140877598153,
                    'Precision': 0.7900677200902935,
                    'Recall': 0.8274231678486997,
                    'AUC': 0.7965831204482793}
}

confusion_matrices = {
    'Logistic Regression': [[306,  91],
                            [75, 348]],
    'SVM': [[253, 144],
            [106, 317]],
    'Random Forest': [[391,   6],
                      [7, 416]],
    'Naive Bayes': [[304,  93],
                    [73, 350]]
}

# Convert metrics to DataFrame
df_metrics = pd.DataFrame(metrics).T
print("Evaluation Metrics:\n")
print(df_metrics)

# Display confusion matrices
print("\nConfusion Matrices:\n")
for model, matrix in confusion_matrices.items():
    print(f"{model}:\n")
    df_matrix = pd.DataFrame(matrix, columns=['Predicted Negative', 'Predicted Positive'], 
                             index=['Actual Negative', 'Actual Positive'])
    print(df_matrix)
    print("\n")


Evaluation Metrics:

                     Accuracy  F1 Score  Precision    Recall       AUC
Logistic Regression  0.797561  0.807425   0.792711  0.822695  0.796738
SVM                  0.695122  0.717195   0.687636  0.749409  0.693344
Random Forest        0.984146  0.984615   0.985782  0.983452  0.984169
Naive Bayes          0.797561  0.808314   0.790068  0.827423  0.796583

Confusion Matrices:

Logistic Regression:

                 Predicted Negative  Predicted Positive
Actual Negative                 306                  91
Actual Positive                  75                 348


SVM:

                 Predicted Negative  Predicted Positive
Actual Negative                 253                 144
Actual Positive                 106                 317


Random Forest:

                 Predicted Negative  Predicted Positive
Actual Negative                 391                   6
Actual Positive                   7                 416


Naive Bayes:

                 Predicted Negativ

In [25]:
#All features 
import pandas as pd

# Data provided
data = {
    'Metric': ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'AUC'],
    'Logistic Regression': ['79.51%', '81.08%', '75.63%', '87.38%', '87.68%'],
    'SVM': ['68.29%', '70.59%', '66.10%', '75.73%', '70.71%'],
    'Random Forest': ['98.54%', '98.52%', '100%', '97.09%', '100%'],
    'Naive Bayes': ['80.00%', '81.78%', '75.41%', '89.32%', '87.06%']
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Set 'Metric' column as the index
df.set_index('Metric', inplace=True)

# Display the table
print(df)


          Logistic Regression     SVM Random Forest Naive Bayes
Metric                                                         
Accuracy               79.51%  68.29%        98.54%      80.00%
F1 Score               81.08%  70.59%        98.52%      81.78%
Precision              75.63%  66.10%          100%      75.41%
Recall                 87.38%  75.73%        97.09%      89.32%
AUC                    87.68%  70.71%          100%      87.06%


In [24]:
# Selected Features 
# Selecting specified features
selected_features = ['age', 'sex', 'chol', 'thalach', 'exang', 'oldpeak', 'ca']
X_train_selected = X_train[selected_features]

# For storing results for the selected features
metrics_selected_features = {}
confusion_matrices_selected_features = {}

# Training models and collecting metrics using the selected features
for name, model in models.items():
    y_pred = cross_val_predict(model, X_train_selected, y_train, cv=5)
    
    accuracy = accuracy_score(y_train, y_pred)
    f1 = f1_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred)
    recall = recall_score(y_train, y_pred)
    auc = roc_auc_score(y_train, y_pred)
    
    metrics_selected_features[name] = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Precision': precision,
        'Recall': recall,
        'AUC': auc
    }
    
    confusion_matrices_selected_features[name] = confusion_matrix(y_train, y_pred)

metrics_selected_features, confusion_matrices_selected_features


({'Logistic Regression': {'Accuracy': 0.7975609756097561,
   'F1 Score': 0.8074245939675174,
   'Precision': 0.7927107061503417,
   'Recall': 0.8226950354609929,
   'AUC': 0.7967379459420834},
  'SVM': {'Accuracy': 0.6951219512195121,
   'F1 Score': 0.7171945701357466,
   'Precision': 0.6876355748373102,
   'Recall': 0.7494089834515366,
   'AUC': 0.6933442902144331},
  'Random Forest': {'Accuracy': 0.9841463414634146,
   'F1 Score': 0.9846153846153846,
   'Precision': 0.985781990521327,
   'Recall': 0.983451536643026,
   'AUC': 0.9841690932585406},
  'Naive Bayes': {'Accuracy': 0.7975609756097561,
   'F1 Score': 0.8083140877598153,
   'Precision': 0.7900677200902935,
   'Recall': 0.8274231678486997,
   'AUC': 0.7965831204482793}},
 {'Logistic Regression': array([[306,  91],
         [ 75, 348]], dtype=int64),
  'SVM': array([[253, 144],
         [106, 317]], dtype=int64),
  'Random Forest': array([[391,   6],
         [  7, 416]], dtype=int64),
  'Naive Bayes': array([[304,  93],
     

In [26]:
# Selected Features 

import pandas as pd

# Data provided
data = {
    'Metric': ['Accuracy', 'F1 Score', 'Precision', 'Recall', 'AUC'],
    'Logistic Regression': ['76.59%', '77.98%', '73.91%', '82.52%', '84.79%'],
    'SVM': ['65.37%', '66.98%', '64.29%', '69.90%', '70.92%'],
    'Random Forest': ['100%', '100%', '100%', '100%', '100%'],
    'Naive Bayes': ['76.10%', '77.83%', '72.88%', '83.50%', '84.39%']
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Set 'Metric' column as the index
df.set_index('Metric', inplace=True)

# Display the table
print(df)


          Logistic Regression     SVM Random Forest Naive Bayes
Metric                                                         
Accuracy               76.59%  65.37%          100%      76.10%
F1 Score               77.98%  66.98%          100%      77.83%
Precision              73.91%  64.29%          100%      72.88%
Recall                 82.52%  69.90%          100%      83.50%
AUC                    84.79%  70.92%          100%      84.39%
